In [1]:
import requests
import pandas as pd
import numpy as np
from ratelimit import limits, RateLimitException, sleep_and_retry
from backoff import on_exception, expo

In [2]:
# CORE API 3.0.0 BETA #

apikey = "ouRpyQ48PL2n0rxTd6SKG3OMWcqCBiJs"

#@on_exception(expo, RateLimitException, max_tries=5)
@sleep_and_retry
@limits(calls=5, period=61)
def query_api(query, scrollId=None):
    
    headers={"Authorization":"Bearer "+apikey}
    search_url = "https://api.core.ac.uk/v3/search/works"
    
    if not scrollId:
        response = requests.get(f"{search_url}?q={query}&limit=50&scroll=true",headers=headers)
    else:
        response = requests.get(f"{search_url}?q={query}&limit=50&scrollId={scrollId}",headers=headers)        
    
    print(response.status_code)
    print(response.headers)
    if response.status_code != 200:
        raise Exception(f"API response: {response.status_code}")
        
    hit = response.json()
    
    for element in hit["results"]: 
        if "fullText" in element:
            del element["fullText"]

    return hit

def iter_query(query):
    allResults = []
    scrollId = None
    
    result = query_api(query)
    
    allResults.append(result["results"])
    scrollId = result["scrollId"]
    while scrollId is not None:
        result = query_api(query, scrollId=scrollId)
        scrollId = result["scrollId"]
        allResults.append(result["results"])

In [ ]:
test = iter_query("katonah+baseball")
test